# RandomOffset

通过添加0到1的随机数将将离散化的数据连续化，由于取整可以得到原数据，并不会破坏数据

## 代码

```c++
#include <TFile.h>
#include <TTree.h>
#include <TString.h>
#include <TRandom3.h>


const int chp = 48;
const int chr = 48;


int main(int argc, char **argv) {
	TFile *ipf = new TFile("../data/s4.root");
	TTree *ipt = (TTree*)ipf->Get("tree");

	Int_t pe[chp], re[chr];

	ipt->SetBranchAddress("pe", pe);
	ipt->SetBranchAddress("re", re);


	TFile *opf = new TFile("../data/s4c.root", "recreate");
	TTree *opt = new TTree("tree", "alpha with random offset");

	TRandom3 *gr = new TRandom3(0);
	Double_t cpe[chp], cre[chr];
	opt->Branch("pe", pe, TString::Format("pe[%d]/I", chp));
	opt->Branch("re", re, TString::Format("re[%d]/I", chr));
	opt->Branch("cpe", cpe, TString::Format("cpe[%d]/D", chp));
	opt->Branch("cre", cre, TString::Format("cre[%d]/D", chr));

	Long64_t nentries = ipt->GetEntries();
	Long64_t nentries100 = nentries / 100;
	printf("randomOffset   0%%");
	for (Long64_t jentry = 0; jentry != nentries; ++jentry) {
		ipt->GetEntry(jentry);
		for (int i = 0; i != chp; ++i) {
			cpe[i] = pe[i] + gr->Uniform(0, 1);
		}
		for (int i = 0; i != chr; ++i) {
			cre[i] = re[i] + gr->Uniform(0, 1);
		}
		opt->Fill();

		if ((jentry+1) % (nentries100) == 0) {
			printf("\b\b\b\b%3lld%%", (jentry+1) / nentries100);
			fflush(stdout);
		}
	}
	printf("\n");

	opt->Write();
	opf->Close();
	ipf->Close();
	return 0;
}

```

## 分析


In [1]:
%jsroot on
TCanvas *c1 = new TCanvas;
TFile *ipf = new TFile("../data/s4.root", "read");
TTree *tree = (TTree*)ipf->Get("tree");
tree->Draw("pe>>(1000, 0, 2000)");
c1->Draw();

In [2]:
tree->Draw("pe[0]>>(1000, 0, 2000)");
c1->Draw();

In [3]:
TFile *ipfc = new TFile("../data/s4c.root", "read");
TTree *iptc = (TTree*)ipfc->Get("tree");
iptc->Print();

******************************************************************************
*Tree    :tree      : alpha with random offset                               *
*Entries :  5790825 : Total =      6671433157 bytes  File  Size = 3047490405 *
*        :          : Tree compression factor =   2.19                       *
******************************************************************************
*Br    0 :pe        : pe[48]/I                                               *
*Entries :  5790825 : Total  Size= 1111923162 bytes  File Size  =   28592019 *
*Baskets :      949 : Basket Size=    2133504 bytes  Compression=  38.89     *
*............................................................................*
*Br    1 :re        : re[48]/I                                               *
*Entries :  5790825 : Total  Size= 1111923162 bytes  File Size  =   41111522 *
*Baskets :      949 : Basket Size=    2133504 bytes  Compression=  27.05     *
*...................................................

In [4]:
iptc->Draw("pe>>(1000, 0, 2000)");
c1->Draw();

In [5]:
iptc->Draw("pe[0]>>(1000, 0, 2000)");
c1->Draw();

In [6]:
iptc->Draw("cpe>>(1000, 0, 2000)");
c1->Draw();

In [7]:
iptc->Draw("cpe[0]>>(1000, 0, 2000)");
c1->Draw();

In [8]:
ipf->Close();
ipfc->Close();